# Reference

Pytorch tutorials : https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

## Collect and tag Data

In [ ]:
from __future__ import print_function , division
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms, utils
from skimage import io , transform
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import os , torch

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion() # interactive

In [ ]:
## read csv file
file_path = 'faces/face_landmarks.csv'
landmarks_frame = pd.read_csv(file_path)

## setting figure which want
number = 65
img_name = landmarks_frame.iloc[number , 0]
landmarks = landmarks_frame.iloc[number, 1:]
landmarks = np.asarray(landmarks)
landmarks = landmarks.astype('float').reshape(-1,2)

## print information 
print('Image name : {}'.format(img_name))
print('Landmarks shape : {}'.format(landmarks.shape))
print('First 4 Landmarks : \n {}'.format(landmarks[:4]))

In [ ]:
def show_landmarks(image , landmarks):
    ## Show image with landmarks ##
    plt.imshow(image)
    plt.scatter(landmarks[ : , 0], landmarks[ : , 1],s=10,marker='.',c='r')
    plt.pause(0.001) # pause a bit so that plots are updated

plt.figure()
image_path = io.imread(os.path.join('faces/',img_name))
show_landmarks(image_path,landmarks)
plt.show()

Dataset class
-------------

``torch.utils.data.Dataset`` is an abstract class representing a
dataset.
Your custom dataset should inherit ``Dataset`` and override the following
methods:

-  ``__len__`` so that ``len(dataset)`` returns the size of the dataset.
-  ``__getitem__`` to support the indexing such that ``dataset[i]`` can
   be used to get $i$\ th sample

Let's create a dataset class for our face landmarks dataset. We will
read the csv in ``__init__`` but leave the reading of images to
``__getitem__``. This is memory efficient because all the images are not
stored in the memory at once but read as required.

Sample of our dataset will be a dict
``{'image': image, 'landmarks': landmarks}``. Our dataset will take an
optional argument ``transform`` so that any required processing can be
applied on the sample. We will see the usefulness of ``transform`` in the
next section.


In [ ]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

Let's instantiate this class and iterate through the data samples. We
will print the sizes of first 4 samples and show their landmarks.

In [ ]:
face_dataset = FaceLandmarksDataset(csv_file=file_path,
                                    root_dir='faces/')

fig = plt.figure()

for i in range(len(face_dataset)):
    sample = face_dataset[i]
    print( " Image number : {} \n image shape : {} \n landmarks shape : {} ".format(i,sample['image'].shape, sample['landmarks'].shape))

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)

    if i == 3:
        plt.show()
        break

Transforms
----------

One issue we can see from the above is that the samples are not of the
same size. Most neural networks expect the images of a fixed size.
Therefore, we will need to write some prepocessing code.
Let's create two transforms:

-  ``Rescale``: to scale the image
-  ``RandomCrop``: to crop from image randomly. This is data
   augmentation.

We will write them as callable classes instead of simple functions so
that parameters of the transform need not be passed everytime it's
called. For this, we just need to implement ``__call__`` method and
if required, ``__init__`` method. We can then use a transform like this:

    tsfm = Transform(params)
    transformed_sample = tsfm(sample)

Observe below how these transforms had to be applied both on the image and
landmarks.


## Rescale

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        ## reference : https://openhome.cc/Gossip/Python/Assert.html 、 https://www.runoob.com/python/python-func-isinstance.html
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        ## reference : https://www.itread01.com/content/1548580867.html
        image, landmarks = sample['image'], sample['landmarks']

        h, w = image.shape[:2] # image.shape : Height x Weight x Color
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'landmarks': landmarks}
    


In [ ]:
scale = Rescale(256)
# Apply each of the above transforms on sample.
fig = plt.figure()
sample = face_dataset[65]
# reference : https://www.runoob.com/python/python-func-enumerate.html
for i, tsfrm in enumerate([scale]):

    transformed_sample = tsfrm(sample)
    ax = plt.subplot(1, 3, i + 1)
    #plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    show_landmarks(**transformed_sample)

plt.show()

## RandomCrop

In [ ]:
class RandomCrop(object):
    """
    Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        seed = 0
        image, landmarks = sample['image'], sample['landmarks']
     
        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(seed, h - new_h)
        left = np.random.randint(seed, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        landmarks = landmarks - [left, top]

        return {'image': image, 'landmarks': landmarks}

In [ ]:
crop = RandomCrop(128)
# Apply each of the above transforms on sample.
fig = plt.figure()
sample = face_dataset[65]
for i, tsfrm in enumerate([crop]):
    transformed_sample = tsfrm(sample)

    ax = plt.subplot(1, 3, i + 1)
    plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    show_landmarks(**transformed_sample)

plt.show()

## Compose Rescale and RandomCrop

Now, we apply the transforms on a sample.

Let's say we want to rescale the shorter side of the image to 256 and
then randomly crop a square of size 224 from it. i.e, we want to compose
``Rescale`` and ``RandomCrop`` transforms.
``torchvision.transforms.Compose`` is a simple callable class which allows us
to do this.

In [ ]:
composed = transforms.Compose([Rescale(256), RandomCrop(224)])
# Apply each of the above transforms on sample.
fig = plt.figure()
sample = face_dataset[65]
for i, tsfrm in enumerate([composed]):
    transformed_sample = tsfrm(sample)

    ax = plt.subplot(1, 3, i + 1)
    plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    show_landmarks(**transformed_sample)

plt.show()

Iterating through the dataset
-----------------------------
First , we need create transforms
-  ``ToTensor``: to convert the numpy images to torch images (we need to
   swap axes).
   


In [ ]:
class ToTensor(object):
    ## Convert ndarrays in sample to tensors. ##
    ## reference : https://blog.csdn.net/xiuxin121/article/details/79082548
    def __call__(self,sample):
        image,landmarks = sample['image'] , sample['landmarks']
        ## swap color axis beause 
        ## numpy , opencv  image : Hight x Width x Color
        ## torch           image : Color x Hight x Width
        image = image.transpose(2,0,1)
        return {'image'    : torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks) }


   Let's put this all together to create a dataset with composed
transforms.
To summarize, every time this dataset is sampled:

-  An image is read from the file on the fly
-  Transforms are applied on the read image
-  Since one of the transforms is random, data is augmentated on
   sampling

We can iterate over the created dataset with a ``for i in range``
loop as before.

In [ ]:
transformed_dataset = FaceLandmarksDataset(csv_file=file_path,
                                           root_dir='faces/',
                                           transform=transforms.Compose([
                                               Rescale(256),
                                               RandomCrop(224),
                                               ToTensor()
                                           ]))

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['image'].size(), sample['landmarks'].size()) 
    if i == 3:
        break

However, we are losing a lot of features by using a simple ``for`` loop to
iterate over the data. In particular, we are missing out on:

-  Batching the data
-  Shuffling the data
-  Load the data in parallel using ``multiprocessing`` workers.

``torch.utils.data.DataLoader`` is an iterator which provides all these
features. Parameters used below should be clear. One parameter of
interest is ``collate_fn``. You can specify how exactly the samples need
to be batched using ``collate_fn``. However, default collate should work
fine for most use cases.


In [ ]:
dataloader = DataLoader(transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=4)

# Helper function to show a batch
def show_landmarks_batch(sample_batched):
    """Show image with landmarks for a batch of samples."""
    images_batch, landmarks_batch = \
            sample_batched['image'], sample_batched['landmarks']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid_border_size = 2
    grid = utils.make_grid(images_batch)
    ## numpy , opencv  image : Hight x Width x Color
    ## torch           image : Color x Hight x Width
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    
    for i in range(batch_size):
        plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size + (i + 1) * grid_border_size,
                    landmarks_batch[i, :, 1].numpy() + grid_border_size,
                    s=10, marker='.', c='r')
        # i * im_size : Counterpoint  
    
        plt.title('Batch from dataloader')

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['landmarks'].size())

    # observe 4th batch and stop.
    if i_batch == 3:
        plt.figure()
        show_landmarks_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break